In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
train_df = pd.read_csv('train.csv')
train_df.set_index('LoanID', inplace=True)

trainX = train_df.drop('Default', axis=1, inplace=False)
trainY = train_df['Default']

display(train_df)

,Age,Income,LoanAmount,CreditScore,MonthsEmployed,NumCreditLines,InterestRate,LoanTerm,DTIRatio,Education,EmploymentType,MaritalStatus,HasMortgage,HasDependents,LoanPurpose,HasCoSigner,Default
LoanID,,,,,,,,,,,,,,,,,
DRIRC89L0T,18,137576,209136,846,26,2,10.47,60,0.81,High School,Self-employed,Single,Yes,No,Business,No,0
TS0FIUNHNU,47,57194,5970,748,30,2,19.72,36,0.73,High School,Unemployed,Divorced,No,Yes,Education,No,0
I0YR284A1V,26,84328,95065,453,7,2,24.25,12,0.45,Master's,Self-employed,Married,No,No,Other,Yes,0
WB1T7NQV8A,53,49795,229582,533,107,3,14.44,60,0.17,Bachelor's,Self-employed,Single,Yes,No,Auto,Yes,1
J6GU9M4G1Z,49,115450,22072,840,0,4,24.48,12,0.11,Bachelor's,Part-time,Single,No,Yes,Education,Yes,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
KYS1HKNGGE,40,116623,161673,651,79,2,23.44,12,0.87,Bachelor's,Part-time,Divorced,No,No,Home,Yes,0
5MAOH3AOZO,67,62958,189499,460,77,3,9.29,36,0.11,Bachelor's,Self-employed,Single,No,No,Business,Yes,0
5Y9Z6NW29X,62,34372,59645,524,94,3,9.72,60,0.24,PhD,Full-time,Single,Yes,No,Auto,No,0


In [5]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

cat_variables = 'Education', 'EmploymentType', 'MaritalStatus', 'HasMortgage', 'HasDependents', 'LoanPurpose', 'HasCoSigner'
num_variables = 'Age', 'Income', 'LoanAmount', 'CreditScore', 'MonthsEmployed', 'NumCreditLines', 'InterestRate', 'LoanTerm', 'DTIRatio',

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_variables),
        ('num', 'passthrough', num_variables)
    ]
)

display(preprocessor.fit_transform(trainX))


array([[ 0.  ,  1.  ,  0.  , ..., 10.47, 60.  ,  0.81],
       [ 0.  ,  1.  ,  0.  , ..., 19.72, 36.  ,  0.73],
       [ 0.  ,  0.  ,  1.  , ..., 24.25, 12.  ,  0.45],
       ...,
       [ 0.  ,  0.  ,  0.  , ...,  9.72, 60.  ,  0.24],
       [ 0.  ,  1.  ,  0.  , ...,  4.31, 48.  ,  0.3 ],
       [ 1.  ,  0.  ,  0.  , ..., 23.14, 24.  ,  0.83]])

In [11]:
import xgboost

from sklearn.model_selection import RandomizedSearchCV

model = xgboost.XGBClassifier()

param_dist = {
    'eta': [0.1, 0.2, 0.3, 0.4, 0.5],
    'gamma': [0.0, 0.2, 0.4, 0.8],
    'max_depth': [3, 6, 12, 18, 24],
}

random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_dist,
    scoring='accuracy',
    n_iter=5,
    cv=5,
    verbose=1,
    n_jobs=-1,
    random_state=42
)


random_search.fit(preprocessor.fit_transform(trainX), trainY)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate...
                                           max_delta_step=None, max_depth=None,
                                           max_leaves=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           multi_strategy=None,
                                           n_estimators=None, n_jobs=None,
                                           num_parallel_tree=None,
                                           random_state=None, ...),
                   n_iter=5, n_jobs=-1,
                   param_distributions={'eta': [0.1, 0.2, 0.3, 0.4, 0.5],
                                        'gamma': [0.0, 0.2, 0.4, 0.8],
                                        'max_depth': [3, 6, 12, 18, 24]},
                   random_state=42, scoring='accuracy', verbose=1)

In [13]:
test_df = pd.read_csv('test.csv')
test_df.set_index('LoanID', inplace=True)
display(test_df)

,Age,Income,LoanAmount,CreditScore,MonthsEmployed,NumCreditLines,InterestRate,LoanTerm,DTIRatio,Education,EmploymentType,MaritalStatus,HasMortgage,HasDependents,LoanPurpose,HasCoSigner
LoanID,,,,,,,,,,,,,,,,
CKV34LU7V7,55,112656,92393,581,113,2,23.54,36,0.15,PhD,Self-employed,Single,Yes,Yes,Home,No
62KTYNH93J,56,91569,131575,641,54,1,15.19,12,0.43,High School,Part-time,Divorced,Yes,Yes,Education,Yes
JGFUSOIUH7,26,78169,75417,569,105,3,18.02,12,0.29,Master's,Part-time,Married,Yes,Yes,Education,Yes
4538THBHOX,26,63033,10804,326,118,1,14.71,24,0.41,High School,Part-time,Single,No,No,Business,Yes
DXLNA06JHR,24,29665,21182,662,102,3,15.02,60,0.69,PhD,Unemployed,Single,No,Yes,Business,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DQRTA8KWGC,51,99473,170353,628,24,1,17.03,12,0.46,PhD,Self-employed,Divorced,Yes,Yes,Auto,Yes
W0FDMPACG3,29,42016,111314,371,51,4,7.10,36,0.50,PhD,Self-employed,Married,No,No,Other,No
MA0F4U8ORY,67,88507,142666,731,51,1,22.89,48,0.79,Bachelor's,Part-time,Divorced,No,No,Education,No


In [16]:
pred = random_search.best_estimator_.predict(preprocessor.transform(test_df))

res = pd.DataFrame(pred, index=test_df.index, columns=['Default'])
res.to_csv('submission.csv')

display(res)


,Default
LoanID,
CKV34LU7V7,0
62KTYNH93J,0
JGFUSOIUH7,0
4538THBHOX,0
DXLNA06JHR,0
...,...
DQRTA8KWGC,0
W0FDMPACG3,0
MA0F4U8ORY,0
